# Import libraries

In [1]:
import os
import warnings

import numpy as np
import pandas as pd

import json

import re

from tqdm import tqdm

from itertools import combinations

In [2]:
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

In [3]:
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)

In [4]:
path_to_save = './../results/03a_features'

# Loading data

In [5]:
path_to_ftir_data= './../data/processed_data/ftir_extracted_features.csv'
path_to_samples_description = './../data/parsed_data/samples_description.csv'

## FTIR-features

In [6]:
initial_features_df = pd.read_csv(path_to_ftir_data, sep=';', index_col=0)

print(initial_features_df.shape)
initial_features_df.head(3)

(75, 40)


,$\overline{I}_{3800\text{–}2200}$,$I_{3733}$,$\overline{I}_{3733 \pm 32}$,$I_{3383}$,$\overline{I}_{3383 \pm 32}$,$I_{3298}$,$\overline{I}_{3298 \pm 32}$,$I_{2930}$,$\overline{I}_{2930 \pm 32}$,$I_{2348}$,$\overline{I}_{2348 \pm 32}$,$\overline{I}_{1800\text{–}900}$,$\overline{I}_{1800\text{–}1500}$,$I_{1722}$,$\overline{I}_{1722 \pm 32}$,$I_{1635}$,$\overline{I}_{1635 \pm 32}$,$I_{1588}$,$\overline{I}_{1588 \pm 32}$,$I_{1547}$,$\overline{I}_{1547 \pm 32}$,$\overline{I}_{1500\text{–}1300}$,$I_{1397}$,$\overline{I}_{1397 \pm 32}$,$\overline{I}_{1300\text{–}900}$,$I_{1243}$,$\overline{I}_{1243 \pm 32}$,$I_{1187}$,$\overline{I}_{1187 \pm 32}$,$I_{1132}$,$\overline{I}_{1132 \pm 32}$,$I_{1075}$,$\overline{I}_{1075 \pm 32}$,$I_{876}$,$\overline{I}_{876 \pm 32}$,"$\Sigma I_{p,\ 3800\text{–}2200}$","$\Sigma I_{p,\ 1800\text{–}900}$","$\Sigma I_{p,\ 1800\text{–}1500}$","$\Sigma I_{p,\ 1500\text{–}1300}$","$\Sigma I_{p,\ 1300\text{–}900}$"
division_1_size_bulk,0.335219,0.050403,0.040558,1.0,0.970571,0.946997,0.941083,0.265624,0.266420,0.179887,0.155429,0.550025,0.550451,0.425727,0.421626,1.0,0.904861,0.810370,0.787646,0.447818,0.476634,0.633387,0.815626,0.780998,0.508124,0.537370,0.528082,0.460874,0.477456,0.633975,0.623632,0.733507,0.719963,0.078995,0.079059,2.442911,5.865267,2.683915,0.815626,2.365726
division_1_size_5,0.364626,0.006416,0.005691,1.0,0.995591,0.909946,0.907866,0.317784,0.311382,0.124856,0.114118,0.494561,0.532219,0.417861,0.413922,1.0,0.909491,0.784825,0.764213,0.404502,0.433921,0.568436,0.739353,0.709564,0.429493,0.497679,0.493277,0.468835,0.482559,0.606951,0.596917,0.633977,0.564463,0.077024,0.079152,2.359002,5.553983,2.607188,0.739353,2.207442
division_1_size_3,0.348870,0.041669,0.030630,1.0,0.981962,0.903540,0.901479,0.430626,0.406201,0.254556,0.218397,0.555338,0.562808,0.382219,0.388092,1.0,0.918714,0.858514,0.836568,0.537376,0.556364,0.577782,0.744396,0.717214,0.538545,0.469872,0.469877,0.550605,0.568517,0.843303,0.820172,0.875649,0.774647,0.138402,0.140829,2.630391,6.261932,2.778108,0.744396,2.739428


## Samples description

In [7]:
samples_description = pd.read_csv(f'{path_to_samples_description}', sep=';', index_col=0)

print(samples_description.shape)
samples_description.head(3)

(75, 6)


,Row_ID,Division,Size,Fraction_hue,Fraction_grouped_hue,Class
division_1_size_bulk,1,1,bulk,$> 0$,$d > 5$,2
division_1_size_5,2,1,5,$< 5$,$2 < d \leq 5$,1
division_1_size_3,3,1,3,$< 3$,$2 < d \leq 5$,1


# Feature engineering

## Function for processing

In [8]:
def generate_transformed_features(data, feature_list, tolerance=50, zero_substitute=1e-8):
    data_transformed = pd.DataFrame(index=data.index)

    def clean(col):
        if col.startswith('$') and col.endswith('$'):
            col = col[1:-1]
        return col.replace("Σ", r"\Sigma")

    def wrap_log(col):
        return fr"$\log {clean(col)}$"

    def wrap_pow(col, power):
        return fr"$({clean(col)})^{{{power}}}$"

    def wrap_div(col1, col2):
        return fr"$\frac{{{clean(col1)}}}{{{clean(col2)}}}$"

    def extract_center(name):
        nums = re.findall(r'\d+', name)
        return int(nums[0]) if nums else None

    def get_interval_tag(center):
        if center is None:
            return None
        if 2200 <= center <= 3800:
            return "high"
        elif 800 <= center <= 1900:
            return "low"
        return None

    def extract_center_and_type(name):
        center = extract_center(name)
        is_avg = r'\overline{I}' in name or '±' in name
        return center, is_avg

    def is_near_duplicate(f1, f2):
        c1, avg1 = extract_center_and_type(f1)
        c2, avg2 = extract_center_and_type(f2)
        if c1 is None or c2 is None:
            return False
        return abs(c1 - c2) <= tolerance and (avg1 or avg2)

    np.seterr(divide='ignore', invalid='ignore')

    for col in feature_list:
        base = data[col].copy()
        base_safe = base.replace(0, zero_substitute)

        data_transformed[wrap_log(col)] = np.log(base_safe)

        for power in [-1]:
            transformed = np.power(base_safe, power)
            transformed.replace([np.inf, -np.inf], np.nan, inplace=True)
            data_transformed[wrap_pow(col, power)] = transformed

    for f1, f2 in combinations(feature_list, 2):
        if is_near_duplicate(f1, f2):
            continue
        c1 = extract_center(f1)
        c2 = extract_center(f2)
        tag1 = get_interval_tag(c1)
        tag2 = get_interval_tag(c2)
        if tag1 != tag2:
            continue

        numerator = data[f1].replace(0, zero_substitute)
        denominator = data[f2].replace(0, zero_substitute)
        ratio = numerator / denominator
        ratio.replace([np.inf, -np.inf], np.nan, inplace=True)

        data_transformed[wrap_div(f1, f2)] = ratio

    return data_transformed

## Calculation process

In [9]:
engineered_features_df = generate_transformed_features(data=initial_features_df, feature_list=initial_features_df.columns, tolerance=50)

print(engineered_features_df.shape)
engineered_features_df.head(3)

(75, 492)


,$\log \overline{I}_{3800\text{–}2200}$,$(\overline{I}_{3800\text{–}2200})^{-1}$,$\log I_{3733}$,$(I_{3733})^{-1}$,$\log \overline{I}_{3733 \pm 32}$,$(\overline{I}_{3733 \pm 32})^{-1}$,$\log I_{3383}$,$(I_{3383})^{-1}$,$\log \overline{I}_{3383 \pm 32}$,$(\overline{I}_{3383 \pm 32})^{-1}$,$\log I_{3298}$,$(I_{3298})^{-1}$,$\log \overline{I}_{3298 \pm 32}$,$(\overline{I}_{3298 \pm 32})^{-1}$,$\log I_{2930}$,$(I_{2930})^{-1}$,$\log \overline{I}_{2930 \pm 32}$,$(\overline{I}_{2930 \pm 32})^{-1}$,$\log I_{2348}$,$(I_{2348})^{-1}$,$\log \overline{I}_{2348 \pm 32}$,$(\overline{I}_{2348 \pm 32})^{-1}$,$\log \overline{I}_{1800\text{–}900}$,$(\overline{I}_{1800\text{–}900})^{-1}$,$\log \overline{I}_{1800\text{–}1500}$,$(\overline{I}_{1800\text{–}1500})^{-1}$,$\log I_{1722}$,$(I_{1722})^{-1}$,$\log \overline{I}_{1722 \pm 32}$,$(\overline{I}_{1722 \pm 32})^{-1}$,$\log I_{1635}$,$(I_{1635})^{-1}$,$\log \overline{I}_{1635 \pm 32}$,$(\overline{I}_{1635 \pm 32})^{-1}$,$\log I_{1588}$,$(I_{1588})^{-1}$,$\log \overline{I}_{1588 \pm 32}$,$(\overline{I}_{1588 \pm 32})^{-1}$,$\log I_{1547}$,$(I_{1547})^{-1}$,$\log \overline{I}_{1547 \pm 32}$,$(\overline{I}_{1547 \pm 32})^{-1}$,$\log \overline{I}_{1500\text{–}1300}$,$(\overline{I}_{1500\text{–}1300})^{-1}$,$\log I_{1397}$,$(I_{1397})^{-1}$,$\log \overline{I}_{1397 \pm 32}$,$(\overline{I}_{1397 \pm 32})^{-1}$,$\log \overline{I}_{1300\text{–}900}$,$(\overline{I}_{1300\text{–}900})^{-1}$,...,$\frac{\overline{I}_{1187 \pm 32}}{I_{1075}}$,$\frac{\overline{I}_{1187 \pm 32}}{\overline{I}_{1075 \pm 32}}$,$\frac{\overline{I}_{1187 \pm 32}}{I_{876}}$,$\frac{\overline{I}_{1187 \pm 32}}{\overline{I}_{876 \pm 32}}$,"$\frac{\overline{I}_{1187 \pm 32}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{1187 \pm 32}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{1187 \pm 32}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{1187 \pm 32}}{\Sigma I_{p,\ 1300\text{–}900}}$",$\frac{I_{1132}}{I_{1075}}$,$\frac{I_{1132}}{\overline{I}_{1075 \pm 32}}$,$\frac{I_{1132}}{I_{876}}$,$\frac{I_{1132}}{\overline{I}_{876 \pm 32}}$,"$\frac{I_{1132}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{I_{1132}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{I_{1132}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{I_{1132}}{\Sigma I_{p,\ 1300\text{–}900}}$",$\frac{\overline{I}_{1132 \pm 32}}{I_{1075}}$,$\frac{\overline{I}_{1132 \pm 32}}{\overline{I}_{1075 \pm 32}}$,$\frac{\overline{I}_{1132 \pm 32}}{I_{876}}$,$\frac{\overline{I}_{1132 \pm 32}}{\overline{I}_{876 \pm 32}}$,"$\frac{\overline{I}_{1132 \pm 32}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{1132 \pm 32}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{1132 \pm 32}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{1132 \pm 32}}{\Sigma I_{p,\ 1300\text{–}900}}$",$\frac{I_{1075}}{I_{876}}$,$\frac{I_{1075}}{\overline{I}_{876 \pm 32}}$,"$\frac{I_{1075}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{I_{1075}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{I_{1075}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{I_{1075}}{\Sigma I_{p,\ 1300\text{–}900}}$",$\frac{\overline{I}_{1075 \pm 32}}{I_{876}}$,$\frac{\overline{I}_{1075 \pm 32}}{\overline{I}_{876 \pm 32}}$,"$\frac{\overline{I}_{1075 \pm 32}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{1075 \pm 32}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{1075 \pm 32}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{1075 \pm 32}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{I_{876}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{I_{876}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{I_{876}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{I_{876}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{\overline{I}_{876 \pm 32}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{876 \pm 32}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{876 \pm 32}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{876 \pm 32}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{\Sigma I_{p,\ 1800\text{–}900}}{\Sigma 

*NaN-values analysis*

In [10]:
nan_df = engineered_features_df.isna().sum()
nan_summary = nan_df[nan_df > 0]
print(nan_summary)

Series([], dtype: int64)


## Merging process

In [11]:
features_df = pd.merge(
    left=initial_features_df,
    right=engineered_features_df,
    how='inner',
    left_index=True,
    right_index=True
)

print(features_df.shape)
features_df.head(3)

(75, 532)


,$\overline{I}_{3800\text{–}2200}$,$I_{3733}$,$\overline{I}_{3733 \pm 32}$,$I_{3383}$,$\overline{I}_{3383 \pm 32}$,$I_{3298}$,$\overline{I}_{3298 \pm 32}$,$I_{2930}$,$\overline{I}_{2930 \pm 32}$,$I_{2348}$,$\overline{I}_{2348 \pm 32}$,$\overline{I}_{1800\text{–}900}$,$\overline{I}_{1800\text{–}1500}$,$I_{1722}$,$\overline{I}_{1722 \pm 32}$,$I_{1635}$,$\overline{I}_{1635 \pm 32}$,$I_{1588}$,$\overline{I}_{1588 \pm 32}$,$I_{1547}$,$\overline{I}_{1547 \pm 32}$,$\overline{I}_{1500\text{–}1300}$,$I_{1397}$,$\overline{I}_{1397 \pm 32}$,$\overline{I}_{1300\text{–}900}$,$I_{1243}$,$\overline{I}_{1243 \pm 32}$,$I_{1187}$,$\overline{I}_{1187 \pm 32}$,$I_{1132}$,$\overline{I}_{1132 \pm 32}$,$I_{1075}$,$\overline{I}_{1075 \pm 32}$,$I_{876}$,$\overline{I}_{876 \pm 32}$,"$\Sigma I_{p,\ 3800\text{–}2200}$","$\Sigma I_{p,\ 1800\text{–}900}$","$\Sigma I_{p,\ 1800\text{–}1500}$","$\Sigma I_{p,\ 1500\text{–}1300}$","$\Sigma I_{p,\ 1300\text{–}900}$",$\log \overline{I}_{3800\text{–}2200}$,$(\overline{I}_{3800\text{–}2200})^{-1}$,$\log I_{3733}$,$(I_{3733})^{-1}$,$\log \overline{I}_{3733 \pm 32}$,$(\overline{I}_{3733 \pm 32})^{-1}$,$\log I_{3383}$,$(I_{3383})^{-1}$,$\log \overline{I}_{3383 \pm 32}$,$(\overline{I}_{3383 \pm 32})^{-1}$,...,$\frac{\overline{I}_{1187 \pm 32}}{I_{1075}}$,$\frac{\overline{I}_{1187 \pm 32}}{\overline{I}_{1075 \pm 32}}$,$\frac{\overline{I}_{1187 \pm 32}}{I_{876}}$,$\frac{\overline{I}_{1187 \pm 32}}{\overline{I}_{876 \pm 32}}$,"$\frac{\overline{I}_{1187 \pm 32}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{1187 \pm 32}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{1187 \pm 32}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{1187 \pm 32}}{\Sigma I_{p,\ 1300\text{–}900}}$",$\frac{I_{1132}}{I_{1075}}$,$\frac{I_{1132}}{\overline{I}_{1075 \pm 32}}$,$\frac{I_{1132}}{I_{876}}$,$\frac{I_{1132}}{\overline{I}_{876 \pm 32}}$,"$\frac{I_{1132}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{I_{1132}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{I_{1132}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{I_{1132}}{\Sigma I_{p,\ 1300\text{–}900}}$",$\frac{\overline{I}_{1132 \pm 32}}{I_{1075}}$,$\frac{\overline{I}_{1132 \pm 32}}{\overline{I}_{1075 \pm 32}}$,$\frac{\overline{I}_{1132 \pm 32}}{I_{876}}$,$\frac{\overline{I}_{1132 \pm 32}}{\overline{I}_{876 \pm 32}}$,"$\frac{\overline{I}_{1132 \pm 32}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{1132 \pm 32}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{1132 \pm 32}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{1132 \pm 32}}{\Sigma I_{p,\ 1300\text{–}900}}$",$\frac{I_{1075}}{I_{876}}$,$\frac{I_{1075}}{\overline{I}_{876 \pm 32}}$,"$\frac{I_{1075}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{I_{1075}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{I_{1075}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{I_{1075}}{\Sigma I_{p,\ 1300\text{–}900}}$",$\frac{\overline{I}_{1075 \pm 32}}{I_{876}}$,$\frac{\overline{I}_{1075 \pm 32}}{\overline{I}_{876 \pm 32}}$,"$\frac{\overline{I}_{1075 \pm 32}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{1075 \pm 32}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{1075 \pm 32}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{1075 \pm 32}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{I_{876}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{I_{876}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{I_{876}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{I_{876}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{\overline{I}_{876 \pm 32}}{\Sigma I_{p,\ 1800\text{–}900}}$","$\frac{\overline{I}_{876 \pm 32}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\overline{I}_{876 \pm 32}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\overline{I}_{876 \pm 32}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{\Sigma I_{p,\ 1800\text{–}900}}{\Sigma I_{p,\ 1800\text{–}1500}}$","$\frac{\Sigma I_{p,\ 1800\text{–}900}}{\Sigma I_{p,\ 1500\text{–}1300}}$","$\frac{\Sigma I_{p,\ 1800\text{–}900}}{\Sigma I_{p,\ 1300\text{–}900}}$","$\frac{\Sigma I_{p,\ 1800\

# Creating features description

## Functions for processing

In [12]:
def classify_feature_type(base_columns, columns):
    base_columns = [col for col in base_columns if "Unnamed" not in col]
    columns = [col for col in columns if "Unnamed" not in col]

    def clean(col):
        if col.startswith('$') and col.endswith('$'):
            col = col[1:-1]
        return col.replace("Σ", r"\Sigma")

    def is_interval(col):
        col = clean(col)
        return r"\overline{I}_{" in col or r"\Sigma I_{p," in col

    def is_peak(col):
        return not is_interval(col)

    def extract_numbers(col):
        return list(map(int, re.findall(r"\d{3,4}", col)))

    def is_in_range(col, low, high):
        return any(low <= num <= high for num in extract_numbers(clean(col)))

    def classify(col, base=True):
        col_clean = clean(col)
        source = "intervals" if is_interval(col_clean) else "peaks"

        if is_in_range(col_clean, 2200, 3800):
            region = "3800_2200"
        elif is_in_range(col_clean, 800, 1900):
            region = "1900_800"
        else:
            region = "unknown"

        kind = "base" if base else "engineered"
        return f"{kind}_features_{source}_{region}"

    groups = {}

    for col in base_columns:
        key = classify(col, base=True)
        groups.setdefault(key, []).append(col)

    for col in columns:
        if col not in base_columns:
            key = classify(col, base=False)
            groups.setdefault(key, []).append(col)

    groups["base_features"] = base_columns
    groups["engineered_features"] = [col for col in columns if col not in base_columns]

    return groups

## Feature groups selection

In [13]:
base_columns = initial_features_df.columns.tolist()
columns = features_df.columns.tolist()

In [14]:
features_description_dict = classify_feature_type(base_columns=base_columns, columns=columns)

In [15]:
for feature_type, feature_lst in features_description_dict.items():
    print(f'(*) {feature_type}: {len(feature_lst)}')

(*) base_features_intervals_3800_2200: 7
(*) base_features_peaks_3800_2200: 5
(*) base_features_intervals_1900_800: 18
(*) base_features_peaks_1900_800: 10
(*) engineered_features_intervals_3800_2200: 64
(*) engineered_features_peaks_3800_2200: 20
(*) engineered_features_intervals_1900_800: 343
(*) engineered_features_peaks_1900_800: 65
(*) base_features: 40
(*) engineered_features: 492


# Saving data

In [16]:
features_df.to_csv('./../data/processed_data/ftir_features_long_list.csv', sep=';')

In [17]:
with open('./../data/processed_data/features_description.json', 'w', encoding='utf-8') as f:
    json.dump(features_description_dict, f, ensure_ascii=False, indent=2)